# Teams and Hotels

Playing around with a backtracking algorithm to solve the problem of assigning teams to hotels during an athletics event. 

In [113]:
import pandas
import numpy as np

In [114]:
teams = pandas.read_csv('teams.csv') \
    .assign(total = lambda df: df['single'] + df['double'])

In [115]:
hotels = pandas.read_csv('hotels.csv', sep=';')

In [116]:
hotels

,name,capacity_single,capacity_double
0,Hotel A,80,50
1,Hotel B,40,20
2,Hotel C,40,20
3,Hotel D,80,55


In [119]:
def solve(hotels, teams):
    # keep track of state in the dataframes themselves0
    teams_hotels = teams \
        .sort_values(['total'], ascending=False) \
        .assign(hotel=None).reset_index(drop=True)
    hotels_and_room_state = hotels \
        .assign(remaining_single = lambda df: df['capacity_single']) \
        .assign(remaining_double = lambda df: df['capacity_double'])
    
    if not place_team(hotels_and_room_state, teams_hotels, 0):
        return 'No solution exists.'
    else:
        return (hotels_and_room_state, teams_hotels)
    
def cycle_hotels(hotels, offset):
    # First hotel is tried first (should be filled to capacity)
    new_indexes = [0]
    # Round-robin through the other hotels
    curr = offset + 1
    for _ in range(len(hotels) - 1):
        new_indexes.append(curr)
        curr += 1
        if curr == len(hotels):
            curr = 1
    return hotels.reindex(new_indexes)
    
def place_team(hotels, teams_hotels, team_idx):
    if team_idx >= len(teams_hotels):
        # Exit condition -- all teams have been placed
        return True
    
    t = teams_hotels.loc[team_idx]

    cycled_hotels = cycle_hotels(hotels, team_idx % len(hotels))

    for hotel_idx, hotel in cycled_hotels.iterrows():
        if can_place_team(t, hotel):
            teams_hotels.loc[team_idx, 'hotel'] = hotel['name']
            hotels.loc[hotel_idx, 'remaining_single'] -= t['single']
            hotels.loc[hotel_idx, 'remaining_double'] -= t['double']
            
            if place_team(hotels, teams_hotels, team_idx + 1):
                return True
            
            # Backtrack
            teams_hotels.loc[team_idx, 'hotel'] = None
            hotels.loc[hotel_idx, 'remaining_single'] += t['single']
            hotels.loc[hotel_idx, 'remaining_double'] += t['double']
            
    return False

def can_place_team(team, hotel):
    return team['hotel'] is None and \
           hotel['remaining_single'] >= team['single'] and \
           hotel['remaining_double'] >= team['double']
    


In [120]:
hotels, teams = solve(hotels, teams)

In [121]:
hotels

,name,capacity_single,capacity_double,remaining_single,remaining_double
0,Hotel A,80,50,0,5
1,Hotel B,40,20,25,5
2,Hotel C,40,20,30,15
3,Hotel D,80,55,50,0


In [122]:
teams

,team,single,double,total,hotel
0,GBR,30,55,85,Hotel D
1,GER,50,10,60,Hotel A
2,HUN,10,35,45,Hotel A
3,SWE,20,0,20,Hotel A
4,NOR,10,10,20,Hotel B
5,FIN,10,5,15,Hotel C
6,ISL,5,5,10,Hotel B
